In [1]:
load('utils.sage') # log2_strict, two_adic_*

We are working over GF(17), so RS codes will only work for block sizes <=17.

In [2]:
F = GF(17)
R.<X> = F[]
EF.<z> = F.extension(X^2 - 3)
F, EF

(Finite Field of size 17, Finite Field in z of size 17^2)

In [3]:
def to_bits(x, n_bits):
    for i in range(n_bits):
        yield (x >> i) & 1
def eq(ys, R=None):
    ys = list(ys)
    if R is None:
        R = PolynomialRing(ys[0].parent(), len(ys), 'x')
    return product(
        x*y + (1 - x)*(1 - y)
        for x, y in zip(R.gens(), ys)
    )
def mle(evals):
    n_bits = log2_strict(len(evals))
    return sum(eq(to_bits(i, n_bits))*evals[i] for i in range(2^n_bits))

# Basic Small-Field Construction
> (..) we allow the polynomial’s coefficient field and the code’s alphabet to be sub-cryptographically sized, though we require that these fields be equal to each other.

In [4]:
def ExtensionCode(f, n, k):
    rs = codes.ReedSolomonCode(f, n, k)
    def encode(v):
        d = v.base_ring().degree()
        if d == 1:
            return rs.encode(v)
        else:
            codewords = [rs.encode(vector([x[i] for x in v])) for i in range(d)]
            return vector([v.base_ring()(list(xs)) for xs in zip(*codewords)])
    return encode

In [5]:
l0, l1 = (1, 3)
l = l0 + l1
m0, m1 = 2^l0, 2^l1
C = ExtensionCode(F, 2^l, m1)

In [6]:
with seed(0):
    evals = [F.random_element() for _ in range(2^l)]
evals

[3, 16, 1, 0, 10, 16, 5, 16, 2, 5, 14, 2, 8, 4, 4, 6]

In [7]:
[mle(evals)(*to_bits(i, 4)) for i in range(16)]

[3, 16, 1, 0, 10, 16, 5, 16, 2, 5, 14, 2, 8, 4, 4, 6]

### Commit to `evals`

In [8]:
t = matrix(m0, m1, evals); t

[ 3 16  1  0 10 16  5 16]
[ 2  5 14  2  8  4  4  6]

In [9]:
u = matrix([C(row) for row in t]); u

[16 11  7 14 11  3  3 10  5 15 11  5  3 10  2  7]
[11  5  5  4  5  5 13 13 11  1  5 12 13  5  4  5]

In [10]:
# todo: merkle commit u

### Open `evals` at `r` = `s`

In [11]:
with seed(1):
    rs = [EF.random_element() for _ in range(l)]
s = mle(evals)(*rs); s
rs, s

([2*z + 11, z + 11, 3*z, 8*z + 10], 11)

In [12]:
r0_expansion = vector([eq(rs[:l1])(*to_bits(i, l1)) for i in range(m1)])
r1_expansion = vector([eq(rs[l1:])(*to_bits(i, l0)) for i in range(m0)])
print(r0_expansion, r1_expansion)
t_prime = r1_expansion * t
t_prime

(z + 6, 11*z + 10, 10*z + 2, 9*z, 12*z + 15, 9*z + 10, 9*z + 1, 7*z + 8) (9*z + 8, 8*z + 10)


(9*z + 10, 14*z + 8, 2*z + 12, 16*z + 3, z + 7, 6*z + 15, 9*z + 12, 5*z + 1)

### Verify

In [13]:
assert t_prime * r0_expansion == s
for j in [randint(0, 2^l - 1) for _ in range(5)]:
    # todo: merkle check u.column(j)
    assert r1_expansion * u.column(j) == C(t_prime)[j]

In [14]:
print(f'proof size: 1 merkle root + ({len(t_prime)*2} + queries * {u.nrows()}) base field elements')

proof size: 1 merkle root + (16 + queries * 2) base field elements


# Block-Level Encoding
> (..) suitable for polynomials over fields smaller than the alphabet of the linear block code selected for use.

In [24]:
def PackedCode(ef, n, k):
    d = ef.degree()
    assert n % d == 0 and k % d == 0
    rs = codes.ReedSolomonCode(ef, n//d, k//d)
    print(rs)
    def encode(v):
        if v.base_ring() is ef:
            return rs.encode(v)
        else:
            assert v.base_ring() is ef.base_ring() and len(v) % d == 0
            packed = vector([ef(v[i:i+d]) for i in range(0, len(v), d)])
            return rs.encode(packed)
    return encode

In [25]:
# since our trace length is 2^5 = 32, we would not be able to apply a code in GF(17)
l0, l1 = (2, 3)
rate_bits = 1

l = l0 + l1
m0, m1 = 2^l0, 2^l1
n = m1 << rate_bits
C = PackedCode(EF, n, m1)
m0, m1, n

[8, 4, 5] Reed-Solomon Code over GF(289)


(4, 8, 16)

In [26]:
with seed(0):
    evals = [F.random_element() for _ in range(2^l)]
print(evals)

[3, 16, 1, 0, 10, 16, 5, 16, 2, 5, 14, 2, 8, 4, 4, 6, 0, 4, 15, 15, 10, 12, 13, 6, 13, 7, 1, 9, 1, 15, 12, 8]


### Commit

In [27]:
t = matrix(m0, m1, evals); print(t)
assert (t.nrows(), t.ncols()) == (m0, m1)

[ 3 16  1  0 10 16  5 16]
[ 2  5 14  2  8  4  4  6]
[ 0  4 15 15 10 12 13  6]
[13  7  1  9  1 15 12  8]


In [28]:
u = matrix([C(row) for row in t]); print(u)
assert (u.nrows(), u.ncols()) == (m0, n//EF.degree())
# todo: merkle commit u

[ 14*z + 2  5*z + 13  13*z + 9   3*z + 1  16*z + 7    z + 15  4*z + 11       4*z]
[       11  12*z + 3         5   3*z + 8     z + 9         5       2*z   5*z + 9]
[  3*z + 4  11*z + 1  7*z + 16   8*z + 8 12*z + 16   3*z + 4 11*z + 15  11*z + 4]
[ 5*z + 10 12*z + 11   5*z + 5   2*z + 4   5*z + 1     z + 7  13*z + 2 13*z + 13]


### Open

In [29]:
with seed(1):
    rs = [EF.random_element() for _ in range(l)]
s = mle(evals)(*rs); s
rs, s

([2*z + 11, z + 11, 3*z, 8*z + 10, 16*z + 3], 2*z + 3)

>  P computes the matrix–vector product t' := ⊗_{i=l1..l-1} (1 − r_i, r_i) · t, here interpreting the
> matrix t as an unpacked, m0 × m1 matrix with entries in Tι. P sends V t' in the clear.

In [30]:
r0_expansion = vector([eq(rs[:l1])(*to_bits(i, l1)) for i in range(m1)])
r1_expansion = vector([eq(rs[l1:])(*to_bits(i, l0)) for i in range(m0)])
print(r0_expansion, r1_expansion)
t_prime = r1_expansion * t
t_prime

(z + 6, 11*z + 10, 10*z + 2, 9*z, 12*z + 15, 9*z + 10, 9*z + 1, 7*z + 8) (7*z + 11, 11*z + 4, 2*z + 14, 14*z + 6)


(4*z, z + 5, z + 11, 8*z, 5*z + 16, 16*z + 8, z + 2, 13*z + 9)

### Verify

Now we reinterpret t_prime in the same way as `ExtensionCode`: as a vector of a higher extension, that is recombined with the extension generator

In [33]:
assert t_prime * r0_expansion == s
for j in [randint(0, n//EF.degree()) for _ in range(10)]:
    # todo: merkle check u.column(j)
    assert r1_expansion * u.column(j) == C(t_prime[::2])[j] + z * C(t_prime[1::2])[j]

In [34]:
print(f'proof size: 1 merkle root + ({len(t_prime)*2} + queries * {2*u.nrows()}) base field elements')

proof size: 1 merkle root + (16 + queries * 8) base field elements


In [ ]:
n//EF.

# FRI (incomplete)

In [27]:
C = PackedCode(EF, n*2//EF.degree(), n)

[16, 8, 9] Reed-Solomon Code over GF(83521)


### normal fri:

In [117]:
EF.<z> = F.extension(X^4 + 3)
n_bits = 3
n = 2^n_bits

In [118]:
with seed(0):
    evals = [F.random_element() for _ in range(n)]
evals

[3, 16, 1, 0, 10, 16, 5, 16]

In [119]:
trace_sg = two_adic_subgroup(F, n_bits)
poly = R.lagrange_polynomial(zip(trace_sg, evals)); poly

3*X^7 + 12*X^6 + 11*X^5 + 7*X^4 + 12*X^3 + 11*X^2 + 13*X + 2

In [120]:
lde_sg = two_adic_subgroup(F, n_bits+1, shift=F.multiplicative_generator())
lde = [poly(x) for x in lde_sg]; lde

[7, 16, 2, 1, 16, 0, 11, 10, 6, 16, 9, 5, 3, 16, 13, 3]

In [121]:
R.lagrange_polynomial(zip(lde_sg, lde)).degree()

7

In [127]:
# DEEP query at p(zeta)
with seed(10):
    zeta = EF.random_element()
zeta, poly(zeta)

(6*z^3 + 15*z^2 + 14*z + 12, 10*z^3 + 5*z^2 + 15*z + 15)

In [128]:
# correct:
quotient = [
    (y - poly(zeta)) / (x - zeta)
    for x, y in zip(lde_sg, lde)
]
PolynomialRing(EF, 'x').lagrange_polynomial(zip(lde_sg, quotient)).degree()
# good degree

6

In [129]:
quotient = [     #  | wrong
                 #  v 
    (y - poly(zeta) + 1) / (x - zeta)
    for x, y in zip(lde_sg, lde)
]
PolynomialRing(EF, 'x').lagrange_polynomial(zip(lde_sg, quotient)).degree()
# bad degree

15

### Packed FRI

**Setup**
- Trace $t \in \mathbb{F}^{n \cdot c}$
- Rate $\rho \in \{2,4,..\}$
- Domain extension $\mathbb{E} = \mathbb{F}[X]/(X^d+..)$ with a subgroup $D$ where $|D| = \rho \cdot n$ and $d$ divides $c$
- Challenge extension $\mathbb{L}$ over $\mathbb{E}$

**Commit**
- $\mathcal{P}$ encodes the columns of $t$ onto $D$ to get the LDE matrix $u \in \mathbb{E}^{\rho \cdot n \times c}$
- $\mathcal{P}$ reinterprets each row of $u$ as $c/d$ chunks of $d$ elements (in $\mathbb{E}$), then reduces each chunk $\sum_{i=0}^{d-1} z^i \cdot x_i$, where $z$ is the root of the irreducible poly of $\mathbb{E}$, to get $\hat{u} \in \mathbb{E}^{\rho \cdot n \times c / d}$, and sends $\mathcal{V}$ a merkle commitment to $\hat{u}$.

**Prove**
- $\mathcal{V}$ sends opening point $x \in \mathbb{L}$ and column combining challenge $\alpha \in \mathbb{L}$
- Considering the polynomials $p_1(x),...,p_c(x)$ that interpolate the columns of $t$, $\mathcal{P}$ wants to show that $p_1(x)=y_1,...$.
    Equivalently, $\mathcal{P}$ will show $g(X) = \sum_{i}^{c} \alpha^i \cdot \frac{p_i(X) - y_i}{X - x}$ is low-degree.
- $\mathcal{P}$ evaluates $p_1(x)=y_1,...$ and sends to $\mathcal{V}$
- $\mathcal{P} and \mathcal{V}$ engage in batched FRI on $g(X)$, with the following modification:
  - At each FRI query point $q \in \mathbb{E}$, in "round 0"
    - Considering $p$ as a vector of polynomials, $\mathcal{V}$ interprets it as $c/d$ chunks, and 

In [134]:
def dft(polys, sg):
    return matrix([[p(x) for x in sg] for p in polys]).T
def idft(m, sg):
    r = PolynomialRing(sg[0].parent(), 'x')
    return [r.lagrange_polynomial(zip(sg, col)) for col in m.columns()]

In [138]:
n_bits = 5
n = 2^n_bits
with seed(0):
    evals = matrix.random(F, n, 8)
print(evals.dimensions(), evals.base_ring())
evals[:4]

(32, 8) Finite Field of size 17


[ 2 14  0 15 11 10 16  2]
[ 9  4 10 14  1 14  3 14]
[12 14  3 13 10  1 14  6]
[ 2 14 13  7  6 14 10  3]

In [139]:
trace_sg = two_adic_subgroup(EF, n_bits)
lde_sg = two_adic_subgroup(EF, n_bits+1, shift=EF.multiplicative_generator())
lde = dft(idft(evals, trace_sg), lde_sg)
print(lde.dimensions(), lde.base_ring())
lde[:4]

(64, 8) Finite Field in z of size 17^4


[ 15*z^3 + 13*z^2 + 6*z + 8       8*z^3 + 10*z^2 + 3*z  6*z^3 + 14*z^2 + 3*z + 11         12*z^3 + 15*z + 12    4*z^3 + 3*z^2 + 3*z + 3   4*z^3 + 9*z^2 + 13*z + 9   4*z^3 + 2*z^2 + 13*z + 9     z^3 + 11*z^2 + 5*z + 4]
[ 3*z^3 + 11*z^2 + 16*z + 1  12*z^3 + 3*z^2 + 3*z + 10   8*z^3 + 8*z^2 + 12*z + 6  2*z^3 + 16*z^2 + 5*z + 10  10*z^3 + 7*z^2 + 11*z + 3        16*z^3 + 16*z^2 + 8         7*z^3 + 7*z^2 + 10   9*z^3 + 15*z^2 + 5*z + 6]
[9*z^3 + 13*z^2 + 15*z + 12 10*z^3 + 10*z^2 + 2*z + 14      z^3 + 12*z^2 + z + 15  12*z^3 + 7*z^2 + 3*z + 11           8*z^2 + 12*z + 9           9*z^2 + 4*z + 10 11*z^3 + 3*z^2 + 14*z + 15  9*z^3 + 9*z^2 + 15*z + 13]
[ 6*z^3 + 9*z^2 + 10*z + 15     z^3 + 2*z^2 + 3*z + 10  15*z^3 + 9*z^2 + 9*z + 11     11*z^3 + z^2 + 2*z + 2        13*z^3 + 5*z^2 + 16    7*z^3 + 7*z^2 + 5*z + 3    9*z^3 + 8*z^2 + 3*z + 8    3*z^3 + 6*z^2 + 4*z + 6]

In [140]:
packed_evals = matrix(EF, [
    [EF(row[i:i+d]) for i in range(0, evals.ncols(), d)]
    for row in evals.rows()
])
print(packed_evals.dimensions(), packed_evals.base_ring())
packed_evals[:4]

(32, 2) Finite Field in z of size 17^4


[         15*z^3 + 14*z + 2 2*z^3 + 16*z^2 + 10*z + 11]
[ 14*z^3 + 10*z^2 + 4*z + 9  14*z^3 + 3*z^2 + 14*z + 1]
[13*z^3 + 3*z^2 + 14*z + 12    6*z^3 + 14*z^2 + z + 10]
[ 7*z^3 + 13*z^2 + 14*z + 2  3*z^3 + 10*z^2 + 14*z + 6]

In [141]:
packed_lde = dft(idft(packed_evals, trace_sg), lde_sg)
packed_lde[:4]

[  6*z^3 + 8*z^2 + 5*z + 16     13*z^3 + 5*z^2 + z + 4]
[11*z^3 + 14*z^2 + 5*z + 11   15*z^3 + 7*z^2 + 4*z + 4]
[ 14*z^3 + 11*z^2 + 5*z + 5           2*z^3 + 13*z + 6]
[  2*z^3 + 7*z^2 + 6*z + 13  12*z^3 + 9*z^2 + 9*z + 10]

Because the code is linear, the packed LDE is a linear combination of the unpacked LDE with powers of the extension ideal.

In [145]:
assert lde[0,0] + z * lde[0,1] + z^2 * lde[0,2] + z^3 * lde[0,3] == packed_lde[0,0]

In [146]:
# DEEP query at p(zeta)
with seed(10):
    zeta = EF.random_element()

In [147]:
# Evaluate each (unpacked) poly at zeta
polys = idft(evals, trace_sg)
p_zeta = [p(zeta) for p in polys]
p_zeta

[2*z^3 + 6*z^2 + 7*z + 3,
 9*z^3 + 13*z^2 + 13*z + 13,
 13*z^3 + 11*z^2 + 13*z + 2,
 4*z^3 + 15*z^2 + 11*z + 2,
 5*z^3 + 13*z^2 + 16,
 5*z^3 + 6*z^2 + 12*z + 2,
 z^3 + 12*z^2 + 7*z + 2,
 12*z^3 + z^2 + 4]

In [148]:
quotient = [
    (y - poly(zeta)) / (x - zeta)
    for x, y in zip(lde_sg, lde)
]

TypeError: unsupported operand parent(s) for -: 'Vector space of dimension 8 over Finite Field in z of size 17^4' and 'Finite Field in z of size 17^4'

In [153]:
RR(X^4+3)

X^4 + 3

In [156]:
z.parent()

Finite Field in z of size 17^4

In [157]:
z.minpoly()

x^4 + 3